In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
# import http.client
# import pycurl 
import time
from scipy.stats import linregress
from datetime import datetime

# Import API key
from config import ATTOM_API

In [39]:
property_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/"

sales_trend = "salestrend/snapshot?geoid="

zipcode = "ZI32608"

sales_trend_dates = "&interval=monthly&startyear=2019&endyear=2020&startmonth=january&endmonth=december"

headers = {'Accept': 'application/json', 'apikey': ATTOM_API}

response = requests.post( f'{property_url}{sales_trend}{zipcode}{sales_trend_dates}', headers = headers)

print(response.status_code)
data = response.json()
data

200


{'status': {'version': '1.0.0',
  'code': 0,
  'msg': 'SuccessWithResult',
  'total': 24},
 'salestrends': [{'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'January 2019',
    'end': 'January 2019',
    'interval': 'monthly'},
   'SalesTrend': {'homesalecount': 50,
    'avgsaleprice': 353490.0,
    'medsaleprice': 153550.0},
   'vintage': {'pubDate': '2021-4-26'}},
  {'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'March 2019',
    'end': 'March 2019',
    'interval': 'monthly'},
   'SalesTrend': {'homesalecount': 42,
    'avgsaleprice': 224033.0,
    'medsaleprice': 163500.0},
   'vintage': {'pubDate': '2021-4-26'}},
  {'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'April 2019',
    'end': 'April 2019',
    'interval': 'monthly'},
   'SalesTrend': {'homesalecount': 95,
    'avgsaleprice': 218566.0,
    'medsaleprice': 184300.0},
   'vintage': {'pubDate': '2021-4-26'}},
  {'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'May 2019',
  

In [68]:
print(data['salestrends'][1]['SalesTrend']['avgsaleprice'])

224033.0


In [71]:
date = []
zip_code = []
total_sales = []
average_price = []
median_price = []

for i in data['salestrends']:
    date.append(i['daterange']['start'])
    zip_code.append(i['location']['geoID'][2:7])
    total_sales.append(i['SalesTrend']['homesalecount'])
    average_price.append(i['SalesTrend']['avgsaleprice'])
    median_price.append(i['SalesTrend']['medsaleprice'])


# sales_data = pd.DataFrame({"Date": city_found, "Country":country, "Latitude":lat, "Longitude":lng, "Max Temperature":maxtemp,
#                                "Humidity":humidity, "Cloudiness":clouds, "Wind Speed": wind_speed,
#                                "Date":date})


str

In [ ]:
date = []
zip_code = []
total_sales = []
average_price = []
median_price = []

for city in cities:
    city_count += 1
    response = requests.get(query_url + city)
    if response.status_code != 200:
        print(f'City is not found, skipping onto next city.')
    else:
        response_json = response.json()
        city_found.append(city)
        lat.append(response_json['coord']['lat'])
        lng.append(response_json['coord']['lon'])
        maxtemp.append(response_json['main']['temp_max'])
        clouds.append(response_json['clouds']['all'])
        humidity.append(response_json['main']['humidity'])
        wind_speed.append(response_json['wind']['speed'])
        country.append(response_json['sys']['country'])
        date.append(response_json['dt'])
        print(f'Processing record {city_count} out of {total_cities}.| City of {city}.')

In [ ]:
city_found.append(city)
        lat.append(response_json['coord']['lat'])
        lng.append(response_json['coord']['lon'])
        maxtemp.append(response_json['main']['temp_max'])
        clouds.append(response_json['clouds']['all'])
        humidity.append(response_json['main']['humidity'])
        wind_speed.append(response_json['wind']['speed'])
        country.append(response_json['sys']['c

In [35]:
# Establish connection to property url
conn = http.client.HTTPSConnection("api.gateway.attomdata.com") 

headers = { 
    'accept': "application/json", 
    'apikey': ATTOM_API, 
} 

conn.request("GET", "/propertyapi/v1.0.0/property/salestrend/snapshot?geoid=ZI32608&interval=monthly&startyear=2019&endyear=2020&startmonth=january&endmonth=december", headers=headers) 
print(conn.request.status_code)

res = conn.getresponse() 
data = res.read() 

# print(data.decode("utf-8"))
# print(res.status_code)

AttributeError: 'function' object has no attribute 'status_code'

In [ ]:
property_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/"





In [ ]:
community_url = "https://api.gateway.attomdata.com/communityapi/v2.0.0"
area = "/area/full?AreaId=CO"



In [ ]:
import http.client 

conn = http.client.HTTPSConnection("api.gateway.attomdata.com") 

headers = { 
    'accept': "application/json", 
    'apikey': "", 
} 

conn.request("GET", "/propertyapi/v1.0.0/property/
detail?address1=4529%20Winona%20Court&address2=
Denver%2C%20CO", headers=headers) 

res = conn.getresponse() 
data = res.read() 

print(data.decode("utf-8"))